In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Kayla_zips_for_Column_Day_test_20191007'

In [2]:
df_stores_test=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Kayla_zips_for_Column_Day_test_20191007/BL 2019Q3 Labor Day IVL4 Test SOTF Stores 8.20.19 from BL.xlsx",
                            sheet_name="Test",dtype=str,skiprows=3)
list_store_ColumbiaDay_test=df_stores_test['Store#'].unique().tolist()
list_store_ColumbiaDay_test=[x for x in list_store_ColumbiaDay_test if x !="nan" ]
len(list_store_ColumbiaDay_test)

30

In [3]:
df_store_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
df_store_zips.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [4]:
df_store_zips=df_store_zips.parse("view_by_store",dtype=str)
df_store_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,nan,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,nan,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [5]:
df_test_stores=df_store_zips[df_store_zips['location_id'].isin(list_store_ColumbiaDay_test)]
df_test_stores.shape,df_test_stores['location_id'].nunique()

((30, 17), 30)

In [6]:
df_test_stores['zips_in_10'].unique()

array(['[]'], dtype=object)

In [7]:
df_CD_test_store_zips=pd.DataFrame()
for ind,row in df_test_stores.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['all_trans_P_zips'])
    S_zips=eval(row['all_trans_S_zips'])
    # no zips in 10
    df_P=pd.DataFrame({"zip_cd":P_zips},index=["P"]*len(P_zips))
    df_S=pd.DataFrame({"zip_cd":S_zips},index=["S"]*len(S_zips))
    df_store=df_P.append(df_S)
    df_store['location_id']=location_id
    df_CD_test_store_zips=df_CD_test_store_zips.append(df_store)

df_CD_test_store_zips=df_CD_test_store_zips.reset_index().rename(columns={"index":"trans_flag"})
df_CD_test_store_zips=df_CD_test_store_zips[['location_id','trans_flag','zip_cd']].reset_index()
del df_CD_test_store_zips['index']

store_center=df_test_stores[["location_id",'zip_cd','latitude_meas','longitude_meas']].rename(columns={"zip_cd":"store_zip"})
df_CD_test_store_zips=pd.merge(df_CD_test_store_zips,store_center,on="location_id",how="left")
df_CD_test_store_zips['latitude_meas']=df_CD_test_store_zips['latitude_meas'].astype(float)
df_CD_test_store_zips['longitude_meas']=df_CD_test_store_zips['longitude_meas'].astype(float)
df_CD_test_store_zips['dist_miles']=df_CD_test_store_zips.apply(lambda x: haversine((x['latitude_meas'],x['longitude_meas']),zip_centers[x['zip_cd']],miles=True),axis=1)


In [8]:
df_CD_test_store_zips_unique=df_CD_test_store_zips[['zip_cd','trans_flag']].drop_duplicates()
df_CD_test_store_zips_unique=df_CD_test_store_zips_unique.sort_values(["zip_cd",'trans_flag']).drop_duplicates("zip_cd")
df_CD_test_store_zips_unique=df_CD_test_store_zips_unique.sort_values(['trans_flag','zip_cd'])

In [9]:
# From Kaylar with removing some stores ealiers by me
whole_us=pd.ExcelFile("./Q3 National Footprint_Less NonMoving Stores.xlsx")
whole_us=whole_us.parse('New Full Footprint - National',dtype=str)
whole_us.shape

(11878, 2)

In [11]:
rest_us=whole_us[~whole_us['zip_cd'].isin(df_CD_test_store_zips_unique['zip_cd'].tolist())]

In [14]:
writer=pd.ExcelWriter("./BL_zips_for_ColumbiaDayTest_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_CD_test_store_zips_unique.to_excel(writer,"ColumbiaDay_TestZip",index=False)
rest_us.to_excel(writer,"balance",index=False)
whole_us.to_excel(writer,"full_footprint",index=False)
df_CD_test_store_zips.to_excel(writer,"Test_Zip_by_Store",index=False)
df_test_stores.to_excel(writer,"ColumbiaDay_stores",index=False)
writer.save()